# Full Pipeline Demo

This notebook demonstrates the complete pipeline workflow:
1. **Full Pipeline** - Run from raw video through all stages
2. **Reuse Outputs** - Use intermediate outputs as inputs to skip stages

## Pipeline Stages
```
Video → Pose Estimation → Postprocessing → Event Extraction → Soundscape
```


In [1]:
import importlib
import torch
import datafawn
importlib.reload(datafawn)

from pathlib import Path

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("CUDA not available, using CPU")


Using GPU: NVIDIA GeForce RTX 5070 Ti Laptop GPU


---
## Setup: Create All Pipeline Components


In [2]:
# =============== POSE ESTIMATOR =============== #
dlc_estimator = datafawn.DeepLabCutPoseEstimator(
    model_name='superanimal_quadruped',
    detector_name='fasterrcnn_resnet50_fpn_v2',
    hrnet_model='hrnet_w32',
    max_individuals=1,
    pcutoff=0.15,
    device=device
)

# =============== POSTPROCESSORS =============== #
rel_paws = ['front_left_paw_rel', 'front_right_paw_rel', 'back_left_paw_rel', 'back_right_paw_rel']
reference_map = {
    'back_base': ['front_left_paw', 'front_right_paw'],
    'tail_base': ['back_left_paw', 'back_right_paw']
}

rel_pp = datafawn.RelativePawPositionPostprocessor()
error_pp = datafawn.ErrorPostprocessor(
    bodyparts=rel_paws,
    use_velocity=True,
    use_likelihood=True,
    use_distance=True,
    velocity_kwargs={'threshold_pixels': 50, 'window_size': 5},
    likelihood_kwargs={'min_likelihood': 0.5},
    distance_kwargs={'reference_map': reference_map, 'max_distance': 300}
)

# =============== EVENT EXTRACTOR =============== #
zeni_extractor = datafawn.ZeniExtractor(
    smooth_window_size=5,
    prominence_percentage=0.05,
    orientation_likelihood_threshold=0.0,
    orientation_smooth_window_size=15,
    show_plots=False
)

# =============== SOUNDSCAPE GENERATOR =============== #
SOUNDSCAPE_CONFIG = {
    'event_sound_map': {
        'front_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398494__anthousai__wind-chimes-single-01.wav'),
        'front_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398493__anthousai__wind-chimes-single-02.wav'),
        'back_left_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398492__anthousai__wind-chimes-single-03.wav'),
        'back_right_paw_strike': Path('data_example/sounds/22415__anthousai__wind-chimes/398496__anthousai__wind-chimes-single-04.wav')
    }
}
ss_generator = datafawn.SoundScapeFromConfig(soundscape_config=SOUNDSCAPE_CONFIG)

print("All components created!")


All components created!


In [3]:
# Create the FULL pipeline with all components
full_pipeline = datafawn.EventDetectionPipeline(
    pose_estimator=dlc_estimator,
    postprocessors=[rel_pp, error_pp],
    event_extractors=[zeni_extractor],
    soundscape_generators=[ss_generator]
)

print("Full pipeline ready!")


Full pipeline ready!


---
## Part 1: Run Full Pipeline from Video

This runs ALL stages:
- ✅ Pose Estimation (DLC)
- ✅ Postprocessing (relative positions + error detection)
- ✅ Event Extraction (Zeni algorithm)
- ✅ Soundscape Generation


In [4]:
# Run full pipeline from raw video
RAW_VIDEO_PATH = 'data_example/raw_videos/deerrunning.mp4'
OUTPUT_DIR = 'full_demo_output/deerrunning'

results = full_pipeline.run(
    video_path=RAW_VIDEO_PATH,
    output_dir=OUTPUT_DIR,
    soundscape_input_video="pose_est"  # Use the labeled video for soundscape
)

print("Full pipeline complete!")


Loading DLC 3.0.0rc13...
Running video inference on [WindowsPath('data_example/raw_videos/deerrunning.mp4')] with superanimal_quadruped_hrnet_w32
Using pytorch for model hrnet_w32
Processing video data_example\raw_videos\deerrunning.mp4
Starting to analyze data_example\raw_videos\deerrunning.mp4
Video metadata: 
  Overall # of frames:    615
  Duration of video [s]:  20.52
  fps:                    29.97
  resolution:             w=640, h=480

Running detector with batch size 1


100%|██████████| 615/615 [00:39<00:00, 15.39it/s]


Running pose prediction with batch size 1


100%|██████████| 615/615 [00:32<00:00, 19.16it/s]


Saving results to full_demo_output\deerrunning\pose_estimation
Saving results in full_demo_output\deerrunning\pose_estimation\deerrunning_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_.h5 and full_demo_output\deerrunning\pose_estimation\deerrunning_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__full.pickle


...

Duration of video [s]: 20.52, recorded with 29.97 fps!
Overall # of frames: 615 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 615/615 [00:05<00:00, 114.82it/s]


Video with predictions was saved as full_demo_output\deerrunning\pose_estimation
📁 Saved pose data: full_demo_output\deerrunning\pose_estimation\pose_data.h5
Dataframe with relative positions created:
Original shape: (615, 117)
New shape: (615, 129)

New bodyparts added:
['back_left_paw_rel', 'back_right_paw_rel', 'front_left_paw_rel', 'front_right_paw_rel']
📁 Saved postprocessed data: full_demo_output\deerrunning\postprocessing\postprocessed_data.h5
📁 Saved events: full_demo_output\deerrunning\events\events.json
Loading video: full_demo_output\deerrunning\pose_estimation\deerrunning_superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2__labeled_before_adapt.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 au

MoviePy - Done.
MoviePy - Writing video full_demo_output\deerrunning\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\deerrunning\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\deerrunning\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\deerrunning\soundscapes\SoundScapeFromConfig_output.mp4
Full pipeline complete!


In [8]:
results.keys()

dict_keys(['pose_data', 'postprocessed_data', 'events', 'soundscapes', 'output_paths', 'metadata'])

In [5]:
# Check what stages ran
print("Stages executed:")
for stage, ran in results['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")

print("\nOutput files created:")
for name, path in results['output_paths'].items():
    print(f"  {name}: {path}")


Stages executed:
  pose_estimation: ✅
  postprocessing: ✅
  event_extraction: ✅
  soundscape: ✅

Output files created:
  pose_estimation: full_demo_output\deerrunning\pose_estimation
  postprocessing: full_demo_output\deerrunning\postprocessing
  events: full_demo_output\deerrunning\events
  soundscapes: full_demo_output\deerrunning\soundscapes
  pose_data_file: full_demo_output\deerrunning\pose_estimation\pose_data.h5
  postprocessed_data_file: full_demo_output\deerrunning\postprocessing\postprocessed_data.h5
  events_file: full_demo_output\deerrunning\events\events.json
  soundscape_SoundScapeFromConfig: full_demo_output\deerrunning\soundscapes\SoundScapeFromConfig_output.mp4


In [9]:
# Check the results
print("Results summary:")
print(f"  Pose data shape: {results['pose_data'].shape}")
print(f"  Postprocessed data shape: {results['postprocessed_data'].shape}")
print(f"\nEvents extracted:")
for (scorer, individual), event_dict in results['events'].items():
    print(f"  {individual}:")
    for event_type, frames in event_dict.items():
        print(f"    {event_type}: {len(frames)} events")


Results summary:
  Pose data shape: (615, 117)
  Postprocessed data shape: (615, 129)

Events extracted:
  animal0:
    front_right_paw_strike: 17 events
    back_left_paw_strike: 16 events
    back_right_paw_strike: 16 events
    front_left_paw_strike: 16 events


---
## Part 2: Reuse Intermediate Outputs

Now let's see how to use outputs from the first run as inputs to skip stages.


### Option A: Start from Pose Data (skip pose estimation)

Use the pose data from the first run. This is useful when:
- Pose estimation already done
- Want to try different postprocessors
- Want to experiment with different event extractors


In [10]:
# Option A1: Use pose_data directly from previous results (in-memory)
results_from_pose = full_pipeline.run(
    pose_data=results['pose_data'],
    output_dir='full_demo_output/from_pose_data_memory',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'  # Need video for soundscape
)

print("From pose_data (in-memory):")
for stage, ran in results_from_pose['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


Dataframe with relative positions created:
Original shape: (615, 117)
New shape: (615, 129)

New bodyparts added:
['back_left_paw_rel', 'back_right_paw_rel', 'front_left_paw_rel', 'front_right_paw_rel']
📁 Saved postprocessed data: full_demo_output\from_pose_data_memory\postprocessing\postprocessed_data.h5
📁 Saved events: full_demo_output\from_pose_data_memory\events\events.json
Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Proc

MoviePy - Done.
MoviePy - Writing video full_demo_output\from_pose_data_memory\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_pose_data_memory\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_pose_data_memory\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_pose_data_memory\soundscapes\SoundScapeFromConfig_output.mp4
From pose_data (in-memory):
  pose_estimation: ⏭️ skipped
  postprocessing: ✅
  event_extraction: ✅
  soundscape: ✅


In [11]:
# Option A2: Use pose_data_path from saved file (from disk)
POSE_DATA_FILE = results['output_paths'].get('pose_data_file')
print(f"Loading from: {POSE_DATA_FILE}")

results_from_pose_file = full_pipeline.run(
    pose_data_path=POSE_DATA_FILE,
    output_dir='full_demo_output/from_pose_data_file',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'
)

print("\nFrom pose_data_path (from file):")
for stage, ran in results_from_pose_file['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


Loading from: full_demo_output\deerrunning\pose_estimation\pose_data.h5
Dataframe with relative positions created:
Original shape: (615, 117)
New shape: (615, 129)

New bodyparts added:
['back_left_paw_rel', 'back_right_paw_rel', 'front_left_paw_rel', 'front_right_paw_rel']
📁 Saved postprocessed data: full_demo_output\from_pose_data_file\postprocessing\postprocessed_data.h5
📁 Saved events: full_demo_output\from_pose_data_file\events\events.json
Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19

MoviePy - Done.
MoviePy - Writing video full_demo_output\from_pose_data_file\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_pose_data_file\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_pose_data_file\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_pose_data_file\soundscapes\SoundScapeFromConfig_output.mp4

From pose_data_path (from file):
  pose_estimation: ⏭️ skipped
  postprocessing: ✅
  event_extraction: ✅
  soundscape: ✅


### Option B: Start from Postprocessed Data (skip pose estimation + postprocessing)

Use postprocessed data. This is useful when:
- Want to try different event extractors only
- Already have postprocessed data from a previous run


In [12]:
# Option B1: Use postprocessed_data directly (in-memory)
results_from_postproc = full_pipeline.run(
    postprocessed_data=results['postprocessed_data'],
    output_dir='full_demo_output/from_postprocessed_memory',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'
)

print("From postprocessed_data (in-memory):")
for stage, ran in results_from_postproc['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


📁 Saved events: full_demo_output\from_postprocessed_memory\events\events.json
Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Processing back_right_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.50s', '2.37s', '3.30s']...
    Sound duration: 4.04 seconds
    Created 16 audio clips for back_right_paw_strike

  Processing front_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.57s', '2.64s', '4.07s']...


MoviePy - Done.
MoviePy - Writing video full_demo_output\from_postprocessed_memory\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_postprocessed_memory\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_postprocessed_memory\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_postprocessed_memory\soundscapes\SoundScapeFromConfig_output.mp4
From postprocessed_data (in-memory):
  pose_estimation: ⏭️ skipped
  postprocessing: ⏭️ skipped
  event_extraction: ✅
  soundscape: ✅


In [13]:
# Option B2: Use postprocessed_data_path from saved file (from disk)
POSTPROC_FILE = results['output_paths'].get('postprocessed_data_file')
print(f"Loading from: {POSTPROC_FILE}")

results_from_postproc_file = full_pipeline.run(
    postprocessed_data_path=POSTPROC_FILE,
    output_dir='full_demo_output/from_postprocessed_file',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'
)

print("\nFrom postprocessed_data_path (from file):")
for stage, ran in results_from_postproc_file['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


Loading from: full_demo_output\deerrunning\postprocessing\postprocessed_data.h5
📁 Saved events: full_demo_output\from_postprocessed_file\events\events.json
Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Processing back_right_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.50s', '2.37s', '3.30s']...
    Sound duration: 4.04 seconds
    Created 16 audio clips for back_right_paw_strike

  Processing front_left_paw_strik

MoviePy - Done.
MoviePy - Writing video full_demo_output\from_postprocessed_file\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_postprocessed_file\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_postprocessed_file\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_postprocessed_file\soundscapes\SoundScapeFromConfig_output.mp4

From postprocessed_data_path (from file):
  pose_estimation: ⏭️ skipped
  postprocessing: ⏭️ skipped
  event_extraction: ✅
  soundscape: ✅


### Option C: Start from Events (soundscape generation only)

Use extracted events. This is useful when:
- Only want to regenerate soundscape with different sounds
- Want to try different soundscape configurations


In [14]:
# Option C1: Use events directly (in-memory)
results_from_events = full_pipeline.run(
    events=results['events'],
    output_dir='full_demo_output/from_events_memory',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'
)

print("From events (in-memory):")
for stage, ran in results_from_events['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Processing back_right_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.50s', '2.37s', '3.30s']...
    Sound duration: 4.04 seconds
    Created 16 audio clips for back_right_paw_strike

  Processing front_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.57s', '2.64s', '4.07s']...
    Sound duration: 5.00 seconds
    Created 16 audio clips for front_left_paw

MoviePy - Done.
MoviePy - Writing video full_demo_output\from_events_memory\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_events_memory\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_events_memory\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_events_memory\soundscapes\SoundScapeFromConfig_output.mp4
From events (in-memory):
  pose_estimation: ⏭️ skipped
  postprocessing: ⏭️ skipped
  event_extraction: ⏭️ skipped
  soundscape: ✅


In [15]:
# Option C2: Use events_path from saved file (from disk)
EVENTS_FILE = results['output_paths'].get('events_file')
print(f"Loading from: {EVENTS_FILE}")

results_from_events_file = full_pipeline.run(
    events_path=EVENTS_FILE,
    output_dir='full_demo_output/from_events_file',
    soundscape_input_video='data_example/raw_videos/deerrunning.mp4'
)

print("\nFrom events_path (from file):")
for stage, ran in results_from_events_file['metadata']['stages_run'].items():
    status = "✅" if ran else "⏭️ skipped"
    print(f"  {stage}: {status}")


Loading from: full_demo_output\deerrunning\events\events.json
Loading video: data_example\raw_videos\deerrunning.mp4
Video FPS: 29.97002997002997
Video duration: 20.52 seconds

Processing individual: animal0 (scorer: superanimal_quadruped_hrnet_w32_fasterrcnn_resnet50_fpn_v2_)

  Processing front_right_paw_strike:
    Found 17 strikes
    Strike timestamps: ['1.50s', '2.57s', '3.47s']...
    Sound duration: 3.16 seconds
    Created 17 audio clips for front_right_paw_strike

  Processing back_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['3.37s', '3.70s', '4.80s']...
    Sound duration: 3.19 seconds
    Created 16 audio clips for back_left_paw_strike

  Processing back_right_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.50s', '2.37s', '3.30s']...
    Sound duration: 4.04 seconds
    Created 16 audio clips for back_right_paw_strike

  Processing front_left_paw_strike:
    Found 16 strikes
    Strike timestamps: ['1.57s', '2.64s', '4.07s']...
    Sound durati

MoviePy - Done.
MoviePy - Writing video full_demo_output\from_events_file\soundscapes\SoundScapeFromConfig_output.mp4



MoviePy - Done !
MoviePy - video ready full_demo_output\from_events_file\soundscapes\SoundScapeFromConfig_output.mp4
Done! Output saved to: full_demo_output\from_events_file\soundscapes\SoundScapeFromConfig_output.mp4
📁 Saved soundscape: full_demo_output\from_events_file\soundscapes\SoundScapeFromConfig_output.mp4

From events_path (from file):
  pose_estimation: ⏭️ skipped
  postprocessing: ⏭️ skipped
  event_extraction: ⏭️ skipped
  soundscape: ✅


---
## Part 3: Save and Load for Later

You can also use `save_results()` and `load_results()` for more control.


In [16]:
# Save all results to a specific directory
saved_paths = full_pipeline.save_results(results, 'full_demo_output/saved_for_later')
print("Saved to:")
for name, path in saved_paths.items():
    print(f"  {name}: {path}")


📁 Saved pose data: full_demo_output\saved_for_later\pose_data.h5
📁 Saved postprocessed data: full_demo_output\saved_for_later\postprocessed_data.h5
📁 Saved events: full_demo_output\saved_for_later\events.json
📁 Saved metadata: full_demo_output\saved_for_later\metadata.json
Saved to:
  pose_data: full_demo_output\saved_for_later\pose_data.h5
  postprocessed_data: full_demo_output\saved_for_later\postprocessed_data.h5
  events: full_demo_output\saved_for_later\events.json
  metadata: full_demo_output\saved_for_later\metadata.json


In [17]:
# Load results back in a new session
loaded = datafawn.EventDetectionPipeline.load_results('full_demo_output/saved_for_later')

print("Loaded:")
print(f"  pose_data: {loaded['pose_data'].shape if 'pose_data' in loaded else 'None'}")
print(f"  postprocessed_data: {loaded['postprocessed_data'].shape if 'postprocessed_data' in loaded else 'None'}")
print(f"  events: {len(loaded.get('events', {}))} individuals")


Loaded:
  pose_data: (615, 117)
  postprocessed_data: (615, 129)
  events: 1 individuals


---
## Summary

| Input Type | Stages Skipped | Stages Run |
|------------|----------------|------------|
| `video_path` | None | Pose Est → Postproc → Events → Soundscape |
| `pose_data` / `pose_data_path` | Pose Est | Postproc → Events → Soundscape |
| `postprocessed_data` / `postprocessed_data_path` | Pose Est, Postproc | Events → Soundscape |
| `events` / `events_path` | Pose Est, Postproc, Events | Soundscape only |

**In-memory vs File:**
- Use `pose_data`, `postprocessed_data`, `events` for in-memory data (same session)
- Use `pose_data_path`, `postprocessed_data_path`, `events_path` for data from files


In [20]:
# Optional: Cleanup demo output
import shutil
if Path('full_demo_output').exists():
    # shutil.rmtree('full_demo_output')  # Uncomment to delete
    print("Demo outputs in 'full_demo_output/' - uncomment above to delete")
